# Poppy Universe – Layer 3: Master Matrix Integration

Welcome to the **Poppy Universe Layer 3 – Master notebook**!  
This notebook integrates the **star, planet, and moon matrix predictions** into a single unified dataset for the recommendation engine. It ensures all user × type scores are combined for Layer 3 semantic ranking.

> Note: This notebook currently uses **simulated user interactions** to merge the matrices.  
> Once we have enough real interactions, the same pipeline will process actual user data for production recommendations.

---

## Goals

1. **Load Layer 3 predictions from all notebooks**  
   - Stars, Planets, and Moons  

2. **Merge predicted matrices into a single user × type DataFrame**  
   - Users in rows, all category values as columns  
   - Fill missing values with 0 or appropriate defaults  

3. **Prepare final CSV for the engine**  
   - Clean schema, sorted columns  
   - Save to Output_Data for consumption by C# engine  

4. **Optional analysis & visualization**  
   - Heatmaps of top types per user  
   - Summary statistics across categories

---

## Folder & File References

- **../../Output_Data/Layer3_Star_Predictions.csv** → Star matrix predictions  
- **../../Output_Data/Layer3_Planet_Predictions.csv** → Planet matrix predictions  
- **../../Output_Data/Layer3_Moon_Predictions.csv** → Moon matrix predictions  
- **../../Output_Data/Layer3_Master_Predictions.csv** → Final merged predictions for engine  
- **Plots/** → Optional heatmaps or visualizations

---

> Note: This notebook focuses **on merging all Layer 3 matrices**. Individual notebooks for Stars, Planets, and Moons remain separate to allow independent updates before final integration.


## 0) Imports

In [9]:
import pandas as pd
import numpy as np
import papermill as pm

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

In [10]:
# --- CONFIG / INPUT ---
# Simulated CSV paths (fallback)
sim_star_path   = 'Files/Layer3_Star_Predictions.csv'
sim_planet_path = 'Files/Layer3_Planet_Predictions.csv'
sim_moon_path   = 'Files/Layer3_Moon_Predictions.csv'

# Backend-provided full interactions table (can be None)
backend_interactions_df = None  # backend sends 1 table with all interactions

# --- MINIMUM REQUIREMENTS ---
MIN_UNIQUE_USERS = 20
MIN_INTERACTIONS_PER_TYPE = 10  # per category type

# --- HELPER FUNCTION TO RUN NOTEBOOKS ---
def run_notebook_with_data(nb_path, interactions_df=None, sim_csv=None, param_name='backend_df'):
    """
    Runs a notebook via papermill, passing the interactions DataFrame as a parameter.
    Only runs if interactions_df meets minimum requirements.
    """
    if interactions_df is not None:
        unique_users = interactions_df['User_ID'].nunique()
        interactions_per_type = interactions_df['Category_Type'].value_counts().min()
        if unique_users >= MIN_UNIQUE_USERS and interactions_per_type >= MIN_INTERACTIONS_PER_TYPE:
            print(f"Running {nb_path} with backend data...")
            pm.execute_notebook(
                nb_path,
                nb_path,  # overwrite notebook
                parameters={param_name: interactions_df}
            )
        else:
            print(f"Skipping {nb_path}; backend data too sparse (users: {unique_users}, min interactions/type: {interactions_per_type})")
    else:
        print(f"Skipping {nb_path}; backend data missing. Using existing CSV ({sim_csv})")

# --- RUN THE COMPONENT NOTEBOOKS (ONLY IF BACKEND DATA IS VALID) ---
run_notebook_with_data('Star_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_star_path)
run_notebook_with_data('Planet_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_planet_path)
run_notebook_with_data('Moon_Matrix_Model.ipynb', interactions_df=backend_interactions_df, sim_csv=sim_moon_path)

# --- LOAD THE FINAL PREDICTIONS ---
star_df   = pd.read_csv(sim_star_path)
planet_df = pd.read_csv(sim_planet_path)
moon_df   = pd.read_csv(sim_moon_path)

Skipping Star_Matrix_Model.ipynb; backend data missing. Using existing CSV (Files/Layer3_Star_Predictions.csv)
Skipping Planet_Matrix_Model.ipynb; backend data missing. Using existing CSV (Files/Layer3_Planet_Predictions.csv)
Skipping Moon_Matrix_Model.ipynb; backend data missing. Using existing CSV (Files/Layer3_Moon_Predictions.csv)


## 2) check Star, Planet, and Moon Predictions

In [11]:
# Quick check
print("Stars:", star_df.shape)
print("Planets:", planet_df.shape)
print("Moons:", moon_df.shape)

Stars: (100, 8)
Planets: (100, 5)
Moons: (100, 8)


In [12]:
star_df.head()

,User_ID,A,B,F,G,K,M,O
0,1,4.497129,4.922089,4.807713,4.891604,4.655937,5.051348,5.057187
1,2,4.935688,4.951473,5.152167,4.825430,5.011024,4.053328,4.980935
2,3,4.825786,5.051594,5.115339,5.090629,4.927838,4.945876,4.977943
3,4,4.813962,4.669691,5.082049,5.120726,4.761617,4.887453,3.851400
4,5,4.991626,4.684849,5.191058,4.909058,4.934418,4.038963,4.075054


In [13]:
planet_df.head()

,User_ID,Dwarf Planet,Gas Giant,Ice Giant,Terrestrial
0,1,5.048112,4.236988,4.639808,5.066884
1,2,5.005370,5.061322,4.844697,4.053726
2,3,4.978769,4.964247,5.023617,4.985488
3,4,4.978829,4.964668,5.023900,4.985611
4,5,4.978868,4.965035,5.024151,4.985740


In [14]:
moon_df.head()

,User_ID,Earth,Jupiter,Mars,Neptune,Pluto,Saturn,Uranus
0,1,4.168995,5.029126,2.283988,5.285286,3.976774,1.785866,2.547727
1,2,3.781905,3.312491,3.773603,3.713574,3.957537,4.458037,4.669255
2,3,5.839388,8.756351,4.964412,4.993010,5.323442,3.375350,3.089129
3,4,4.491764,3.895327,4.050290,4.837205,4.683720,4.819854,5.293635
4,5,4.659354,4.057762,5.116182,4.141539,4.906996,6.051296,6.088886


## 3) Merge Star, Planet, and Moon matrices

In [15]:
# Drop User_ID from planet and moon so it doesn't duplicate
planet_values = planet_df.drop('User_ID', axis=1)
moon_values = moon_df.drop('User_ID', axis=1)

# Concatenate horizontally
merged_df = pd.concat([star_df, planet_values, moon_values], axis=1)

# Optional: reorder columns so User_ID is first
cols = ['User_ID'] + [c for c in merged_df.columns if c != 'User_ID']
merged_df = merged_df[cols]

# Quick check
print("Merged shape:", merged_df.shape)
merged_df.head()

Merged shape: (100, 19)


,User_ID,A,B,F,G,K,M,O,Dwarf Planet,Gas Giant,Ice Giant,Terrestrial,Earth,Jupiter,Mars,Neptune,Pluto,Saturn,Uranus
0,1,4.497129,4.922089,4.807713,4.891604,4.655937,5.051348,5.057187,5.048112,4.236988,4.639808,5.066884,4.168995,5.029126,2.283988,5.285286,3.976774,1.785866,2.547727
1,2,4.935688,4.951473,5.152167,4.825430,5.011024,4.053328,4.980935,5.005370,5.061322,4.844697,4.053726,3.781905,3.312491,3.773603,3.713574,3.957537,4.458037,4.669255
2,3,4.825786,5.051594,5.115339,5.090629,4.927838,4.945876,4.977943,4.978769,4.964247,5.023617,4.985488,5.839388,8.756351,4.964412,4.993010,5.323442,3.375350,3.089129
3,4,4.813962,4.669691,5.082049,5.120726,4.761617,4.887453,3.851400,4.978829,4.964668,5.023900,4.985611,4.491764,3.895327,4.050290,4.837205,4.683720,4.819854,5.293635
4,5,4.991626,4.684849,5.191058,4.909058,4.934418,4.038963,4.075054,4.978868,4.965035,5.024151,4.985740,4.659354,4.057762,5.116182,4.141539,4.906996,6.051296,6.088886


## 4) Save final merged matrix for engine

In [16]:
# -----------------------------
# Save final merged matrix for engine
# -----------------------------
merged_df.to_csv('../../Output_Data/Layer_3_Final_Predictions.csv', index=False)
print("Saved Layer3_Final_Predictions.csv")


Saved Layer3_Final_Predictions.csv
